In [14]:
# Import relevant libraries
import os
import pandas as pd
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT
from SALib.sample import saltelli
import json
from ModularCirc import BatchRunner
from comparative_gsa.simulate_data import simulate_data
from comparative_gsa.calculate_output_features import calculate_output_features
from SALib.analyze.sobol import analyze

In [15]:
param_path = '../inputs/parameters_naghavi_constrained_fixed_T_v_tot_v_ref.json'
# Get the filename from the path, without extension
param_filename = os.path.splitext(os.path.basename(param_path))[0]

In [16]:
# Set up the batch runner

## read in save parameters to the folder where simulations r saved.
with open(param_path, 'r') as f:
    params = json.load(f)

# Set up the batch runner
br = BatchRunner('Sobol', 0) # why are we using 'Sobol' here?
br.setup_sampler(param_path)
br.sample(10**6) # setting to a large number just to have more than n_samples. It will later get truncated down to n_samples

/Users/lbokeria/projects/health_gc/cvd_net_project/comparative-gsa/venv/lib/python3.12/site-packages/ModularCirc/_BatchRunner.py:59: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples  = self._sampler.random(nsamples)


In [17]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.t_tr,la.tau,la.t_max,lv.v_ref,lv.delay,lv.t_tr,lv.tau,lv.t_max,T,v_tot
0,305.576111,0.364705,1047.778645,2.796825,7.379755,116.738799,5.767604,3.872886,0.253016,0.650535,...,225,25,150,10,0,420,25,280,800,5000
1,224.490389,0.267449,1391.318553,3.568193,9.100033,91.786620,6.305379,4.954608,0.372932,0.446568,...,225,25,150,10,0,420,25,280,800,5000
2,254.249478,0.327790,906.121586,2.608205,10.275423,104.035536,5.166425,4.290832,0.334445,0.542466,...,225,25,150,10,0,420,25,280,800,5000
3,271.680109,0.230533,1251.458715,3.003151,8.699872,129.094902,6.925526,3.321711,0.286747,0.613231,...,225,25,150,10,0,420,25,280,800,5000
4,285.041178,0.313907,1283.209757,3.239476,7.965886,122.133654,4.714354,4.508952,0.315288,0.459171,...,225,25,150,10,0,420,25,280,800,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,286.958715,0.270841,1193.099273,3.187537,9.940166,92.455483,6.650155,4.455353,0.336288,0.551559,...,225,25,150,10,0,420,25,280,800,5000
999996,269.825749,0.347428,1369.886897,3.048864,9.540161,99.420076,5.763156,3.088968,0.271051,0.430619,...,225,25,150,10,0,420,25,280,800,5000
999997,252.359658,0.294685,1007.688464,2.566349,7.749278,131.007252,6.350748,4.169229,0.317104,0.639150,...,225,25,150,10,0,420,25,280,800,5000
999998,226.634369,0.310516,1230.011513,3.610290,8.822532,118.817102,5.241182,5.043211,0.353617,0.607705,...,225,25,150,10,0,420,25,280,800,5000


In [18]:
br._parameters_2_sample

{'ao.r': (195.0, 325.0),
 'ao.c': (0.22499999999999998, 0.375),
 'art.r': (843.75, 1406.25),
 'art.c': (2.25, 3.75),
 'ven.r': (6.75, 11.25),
 'ven.c': (84.975, 141.625),
 'av.r': (4.5, 7.5),
 'mv.r': (3.0749999999999997, 5.125),
 'la.E_pas': (0.22499999999999998, 0.375),
 'la.E_act': (0.41250000000000003, 0.6875),
 'la.k_pas': (0.01665, 0.07500000000000001),
 'lv.E_pas': (0.75, 1.25),
 'lv.E_act': (2.25, 3.75),
 'lv.k_pas': (0.00999, 0.045)}

In [19]:
# Now, generate samples for sensitivity analysis by taking the keys of _parameters_2_sample and turning into a list
relevant_columns = list(br._parameters_2_sample.keys())
relevant_columns

['ao.r',
 'ao.c',
 'art.r',
 'art.c',
 'ven.r',
 'ven.c',
 'av.r',
 'mv.r',
 'la.E_pas',
 'la.E_act',
 'la.k_pas',
 'lv.E_pas',
 'lv.E_act',
 'lv.k_pas']

In [20]:
problem = {
    'num_vars': len(relevant_columns),
    'names': relevant_columns,
    'bounds' : br.samples[relevant_columns].describe().loc[['min', 'max']].T.values
}

In [21]:
param_values = saltelli.sample(problem, 16, calc_second_order=True)
param_values.shape

/var/folders/rc/594kgpld1czfpr55m3_m1szw0000gr/T/ipykernel_71233/3559648656.py:1: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 16, calc_second_order=True)


(480, 14)

In [22]:
param_values = pd.DataFrame(param_values) 

param_values.columns = relevant_columns

param_values

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,la.k_pas,lv.E_pas,lv.E_act,lv.k_pas
0,207.187532,0.295313,1107.421704,3.234375,8.015627,139.854669,6.093751,4.804687,0.295312,0.455469,0.022120,0.953125,3.234375,0.032965
1,239.687522,0.295313,1107.421704,3.234375,8.015627,139.854669,6.093751,4.804687,0.295312,0.455469,0.022120,0.953125,3.234375,0.032965
2,207.187532,0.229688,1107.421704,3.234375,8.015627,139.854669,6.093751,4.804687,0.295312,0.455469,0.022120,0.953125,3.234375,0.032965
3,207.187532,0.295313,1283.202724,3.234375,8.015627,139.854669,6.093751,4.804687,0.295312,0.455469,0.022120,0.953125,3.234375,0.032965
4,207.187532,0.295313,1107.421704,3.140625,8.015627,139.854669,6.093751,4.804687,0.295312,0.455469,0.022120,0.953125,3.234375,0.032965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,280.312510,0.351562,966.796887,2.859375,9.421876,125.692185,5.718751,3.523440,0.229688,0.489844,0.018473,1.046875,2.390625,0.011084
476,280.312510,0.351562,966.796887,2.859375,9.421876,125.692185,5.718751,3.523440,0.229688,0.489844,0.065883,0.921875,2.390625,0.011084
477,280.312510,0.351562,966.796887,2.859375,9.421876,125.692185,5.718751,3.523440,0.229688,0.489844,0.065883,1.046875,3.703125,0.011084
478,280.312510,0.351562,966.796887,2.859375,9.421876,125.692185,5.718751,3.523440,0.229688,0.489844,0.065883,1.046875,2.390625,0.017648


In [23]:
# How many rows are in param_values
param_values.shape[0]

# Truncate br.samples to be only the first n_samples rows
br._samples = br._samples.iloc[:param_values.shape[0]].copy()

In [24]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.t_tr,la.tau,la.t_max,lv.v_ref,lv.delay,lv.t_tr,lv.tau,lv.t_max,T,v_tot
0,305.576111,0.364705,1047.778645,2.796825,7.379755,116.738799,5.767604,3.872886,0.253016,0.650535,...,225,25,150,10,0,420,25,280,800,5000
1,224.490389,0.267449,1391.318553,3.568193,9.100033,91.786620,6.305379,4.954608,0.372932,0.446568,...,225,25,150,10,0,420,25,280,800,5000
2,254.249478,0.327790,906.121586,2.608205,10.275423,104.035536,5.166425,4.290832,0.334445,0.542466,...,225,25,150,10,0,420,25,280,800,5000
3,271.680109,0.230533,1251.458715,3.003151,8.699872,129.094902,6.925526,3.321711,0.286747,0.613231,...,225,25,150,10,0,420,25,280,800,5000
4,285.041178,0.313907,1283.209757,3.239476,7.965886,122.133654,4.714354,4.508952,0.315288,0.459171,...,225,25,150,10,0,420,25,280,800,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,291.209526,0.295114,973.306977,3.562147,7.871498,123.096524,5.967815,3.722207,0.231974,0.418152,...,225,25,150,10,0,420,25,280,800,5000
476,273.827115,0.361754,1009.602669,3.701109,7.102280,130.296101,6.397712,5.101378,0.370644,0.572481,...,225,25,150,10,0,420,25,280,800,5000
477,256.238272,0.271565,1367.973253,2.683879,9.886172,105.676072,5.529236,4.004640,0.250324,0.505747,...,225,25,150,10,0,420,25,280,800,5000
478,222.502074,0.323673,869.592642,3.139686,11.238445,89.710633,7.371338,3.150921,0.289181,0.465706,...,225,25,150,10,0,420,25,280,800,5000


In [25]:
n_samples = param_values.shape[0]

simulation_out_path = f'../outputs/simulations/output_{n_samples}_samples_{param_filename}/'

# Make this directory if it doesn't exist
os.makedirs(simulation_out_path, exist_ok=True)

In [26]:
# Now finish setting up the batch runner
map_ = {
    'lv.t_tr' : ['lv.t_tr',],
    'la.t_tr' : ['la.t_tr',],
    'la.delay' : ['la.delay',],
    'lv.tau' : ['lv.tau',],
    'la.tau' : ['la.tau',],
    'lv.t_max' : ['lv.t_max',],
    'la.t_max' : ['la.t_max',],
}

# Map the sample timings
br.map_sample_timings(
    ref_time=1000., # double check if 1000 or 1
    map=map_
    )

In [27]:
# Map the vessel volumes
br.map_vessel_volume()


# Save the samples to a CSV file
br.samples.to_csv(os.path.join(simulation_out_path,
                                f'input_samples_{n_samples}.csv'),
                                index=False)

# Set up the model with the parameters and time setup
br.setup_model(model=NaghaviModel, po=NaghaviModelParameters,
                time_setup=TEMPLATE_TIME_SETUP_DICT)

In [28]:
simulations, bool_indices = simulate_data(
    batch_runner=br,
    simulation_out_path=simulation_out_path
)

Running batch simulation with 480 samples...


100%|██████████| 480/480 [00:46<00:00, 10.43it/s]


[]


In [29]:
summary_df = calculate_output_features(
        simulations=simulations,
        simulation_out_path=simulation_out_path)

Saved as simulation_summary.csv
